In [1]:
'''import libraries used, json to read the data, 
randint for randomly generated paragraphs, 
and pandas to create a dataframe to be written to csv'''
import json
import pandas as pd
from random import randint
import string
# Importing the NLTK library
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import spacy
from nltk.stem.wordnet import WordNetLemmatizer

punc = string.punctuation  

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
 
# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Opening JSON file, with specific file name
fname = 'train-SQuAD-v2.0'
f = open(fname + '.json')
#load in data as python dictionary using the json library
data = json.load(f)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\a1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\a1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#access the data key in the dictionary for access to all question frameworks
data = data['data']

In [3]:
def nltk_POS(text):
    words = word_tokenize(text)
    # Performing PoS tagging
    pos_tags = pos_tag(words)
    sent = []
    for word, tag in pos_tags:
        if(word == "not" or word == "n't"):
            sent.append("NOT")
        else:
            sent.append(tag)
    return [words, sent]

In [11]:
print(nltk_POS("hadn't"))

[['had', "n't"], ['VBD', 'NOT']]


In [78]:
add_not = ["VB", "VBP", "VBZ", "MD","VBN", "VBG", "VBD"]
add_not_special = ["VBN", "VBG", "VBD", "VB"]
special_words = ["is", "did", "do", "does", "was", "has", "should", "could", "would", "must", "am", "are", "were", "was", "have", "had"]

In [82]:
WordNetLemmatizer().lemmatize("makes", 'v')

'make'

In [134]:
#this is the dataframe that will be converted to pandas and then to csv
nltk_df = []
counter = 0
#loop through all different data titles/topics associated!
for i in range(0, len(data)):
    #get number of paragraphs per topic
    length = len(data[i]['paragraphs'])
    #get list of all paragraphs for later use
    paragraphs = data[i]['paragraphs']
    #loop through each paragraph
    for j in range(0, len(paragraphs)):
        #get all of the questions associated per paragraph
        questions = paragraphs[j]['qas']
        #get the context paragraph value
        context = paragraphs[j]['context']
        #loop through all questions in each topic paragraph
        for k in range(0, len(questions)):
            if(questions[k]['is_impossible'] == False):
                #get the question id
                id = questions[k]['id']
                #get question value
                question = questions[k]['question']
                
                #add dictionary row value as per columns in instruction(assume modified is the same as original question, no description otherwise)
                nltk_mod = nltk_POS(question)
                sent = ""
                if("NOT" in nltk_mod[1]):
                    for z in range(0, len(nltk_mod[0])):
                        if(nltk_mod[1][z] != "NOT"):
                            sent += nltk_mod[0][z]
                            if(z != len(nltk_mod[0]) - 1):
                                if(nltk_mod[0][z + 1] not in punc):
                                    sent += " "
                    nltk_row = {"Question_Id": id, "Original_Question": question, "Original_Passage": context, "Modified_Question": sent}
                    nltk_df.append(nltk_row)
                else:
                    sent_cases = []
                    cases = 0
                    is_not = False
                    for z in range(0, len(nltk_mod[0])):
                        #not_added = nltk_mod[0][z]
                        if(nltk_mod[1][z] in add_not):
                            cases += 1
                            
                    for z in range(0, cases):
                        is_checked = False
                        sent = ""
                        for n in range(0, len(nltk_mod[0])):
                            not_added = nltk_mod[0][n]
                            if(nltk_mod[1][n] in add_not):
                                if(is_checked == False):
                                    if(nltk_mod[0][n] in special_words):
                                        not_added = nltk_mod[0][n] + "n't"
                                    elif(nltk_mod[0][n] == "will"):
                                        not_added = "won't" 
                                    elif(nltk_mod[0][n] == "can"):
                                        not_added = "can't"
                                    else:
                                        if(nltk_mod[1][n] == "VBZ" or nltk_mod[1][n] == "VBD"):
                                            if(nltk_mod[0][n-1] != "was"):
                                                not_added = "didn't " + WordNetLemmatizer().lemmatize(nltk_mod[0][n], 'v')
                                        else:
                                            if(nltk_mod[1][n] == "VB" or nltk_mod[1][n] == "VBG" or nltk_mod[1][n] == "VBN"):
                                                not_added = "not " + nltk_mod[0][n] #+ WordNetLemmatizer().lemmatize(nltk_mod[0][n], 'v')
                                            else:
                                                not_added += " not"
                                    nltk_mod[1][n] = "NULL"
                                    is_checked = True
                            else:
                                if(is_checked == False):
                                    if(n < len(nltk_mod[0]) - 2):
                                        if(nltk_mod[1][n+1] == "TO" and nltk_mod[1][n] == "NN" and nltk_mod[1][n+2] in add_not):
                                            not_added = "not " + nltk_mod[0][n]
                                            nltk_mod[1][n+2] = "NULL"
                                            is_checked = True
                            if(n != len(nltk_mod[0]) - 1):
                                if((nltk_mod[0][n + 1] not in punc) and (nltk_mod[0][n + 1] != "'s")):
                                   not_added += " "
                            sent += not_added
                        sent_cases.append(sent)
                                
                    for case in sent_cases:
                        nltk_row = {"Question_Id": id, "Original_Question": question, "Original_Passage": context, "Modified_Question": case}
                        nltk_df.append(nltk_row)

                counter += 1
                if(counter%1000 == 0):
                    print(str(counter) + " out of: 131,000")
            else:
                pass
            

1000 out of: 131,000
2000 out of: 131,000
3000 out of: 131,000
4000 out of: 131,000
5000 out of: 131,000
6000 out of: 131,000
7000 out of: 131,000
8000 out of: 131,000
9000 out of: 131,000
10000 out of: 131,000
11000 out of: 131,000
12000 out of: 131,000
13000 out of: 131,000
14000 out of: 131,000
15000 out of: 131,000
16000 out of: 131,000
17000 out of: 131,000
18000 out of: 131,000
19000 out of: 131,000
20000 out of: 131,000
21000 out of: 131,000
22000 out of: 131,000
23000 out of: 131,000
24000 out of: 131,000
25000 out of: 131,000
26000 out of: 131,000
27000 out of: 131,000
28000 out of: 131,000
29000 out of: 131,000
30000 out of: 131,000
31000 out of: 131,000
32000 out of: 131,000
33000 out of: 131,000
34000 out of: 131,000
35000 out of: 131,000
36000 out of: 131,000
37000 out of: 131,000
38000 out of: 131,000
39000 out of: 131,000
40000 out of: 131,000
41000 out of: 131,000
42000 out of: 131,000
43000 out of: 131,000
44000 out of: 131,000
45000 out of: 131,000
46000 out of: 131,0

In [135]:
#create dataframes from the 
nltk_df = pd.DataFrame(nltk_df)

In [136]:
#display this df and one below to ensure correct outputs
nltk_df.head(21)

,Question_Id,Original_Question,Original_Passage,Modified_Question
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When didn't Beyonce start becoming popular?
1,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce not start becoming popular?
2,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start not becoming popular?
3,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas didn't Beyonce compete in when she ...
4,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...
5,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...
6,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When didn't Beyonce leave Destiny's Child and ...
7,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce not leave Destiny's Child and...
8,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and not...
9,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state didn't Beyonce grow up?


In [137]:
nltk_df.to_csv("negation_handling.csv")